In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [111]:
df = pd.read_csv('/Volumes/GoogleDrive/My Drive/Spring 19/CSE 494 Labs/Homework 1/hw1_md5_list.csv')

In [112]:
df.head()

Unnamed: 0                        CTB-locker  \
0           0  c4b29d3345ed0b7a0b284764f3446396   
1           1  aadf8bb8770d563aa27eadc57ef238e9   
2           2  db0a2be5b0eb4603fada6e6f79f3d267   
3           3  3be529d7c05913ef0571a6f4fbf7d946   
4           4  555eaa028a55704f3200614a01d0a464   

                          cybergate                            dridex  \
0  e6b37fe5775812b5006cb6a2bb89977c  9e06e2fcda95624e24207112632934de   
1  31fab5d4dc9e7baba161e6eca79af442  5b4c1b0e5c0c375594018a79b2a7a4be   
2  50198b5b3b60bed98a6531fbed2252a0  a78a2cfcb76422b16936537bc5296996   
3  532265c36a89daeb95a025395a817348  b382486264996132983c279b056bab2a   
4  d3cce60e0f2b7d92ccbfc9dc1bd7ad0f  dfa8814b043cbd03bbd56fe6783a159d   

                              locky                        Teslacrypt  \
0  297529814d8d292594a1981fad30daa6  a30863f1a404bc2f735cc9ad862e85a9   
1  b97ed89e814ad91338a6bdd5f7853566  a2fb8550db4d13c218f98862e24ef105   
2  40849d82a14058cbc91d0ecea473d1d8  a1559181f4d306118f589dd86a8a3c30   
3  254c5c13dd02a9fccbe2a40ac0b04355  2180152ae725f04088dfc9eae68066a6   
4  1d0687fe7c7c5591f1049ecb84e8cbd9  a448ffac07a217b95d4c6478c487f096   

                             xtreme  
0  00047fb702c1bd3b20d2254650df3768  
1  012ef682f57c5b242736fdbf533216ce  
2  03afa299cdd5f04260c3c802a08e151d  
3  03ccd72bae83487eea6585340c1f54ad  
4  053c17c285aafa964535325a00eb934b

In [113]:
df = df.iloc[:,1:]
df.head()

CTB-locker                         cybergate  \
0  c4b29d3345ed0b7a0b284764f3446396  e6b37fe5775812b5006cb6a2bb89977c   
1  aadf8bb8770d563aa27eadc57ef238e9  31fab5d4dc9e7baba161e6eca79af442   
2  db0a2be5b0eb4603fada6e6f79f3d267  50198b5b3b60bed98a6531fbed2252a0   
3  3be529d7c05913ef0571a6f4fbf7d946  532265c36a89daeb95a025395a817348   
4  555eaa028a55704f3200614a01d0a464  d3cce60e0f2b7d92ccbfc9dc1bd7ad0f   

                             dridex                             locky  \
0  9e06e2fcda95624e24207112632934de  297529814d8d292594a1981fad30daa6   
1  5b4c1b0e5c0c375594018a79b2a7a4be  b97ed89e814ad91338a6bdd5f7853566   
2  a78a2cfcb76422b16936537bc5296996  40849d82a14058cbc91d0ecea473d1d8   
3  b382486264996132983c279b056bab2a  254c5c13dd02a9fccbe2a40ac0b04355   
4  dfa8814b043cbd03bbd56fe6783a159d  1d0687fe7c7c5591f1049ecb84e8cbd9   

                         Teslacrypt                            xtreme  
0  a30863f1a404bc2f735cc9ad862e85a9  00047fb702c1bd3b20d2254650df3768  
1  a2fb8550db4d13c218f98862e24ef105  012ef682f57c5b242736fdbf533216ce  
2  a1559181f4d306118f589dd86a8a3c30  03afa299cdd5f04260c3c802a08e151d  
3  2180152ae725f04088dfc9eae68066a6  03ccd72bae83487eea6585340c1f54ad  
4  a448ffac07a217b95d4c6478c487f096  053c17c285aafa964535325a00eb934b

In [114]:
def load_requests(md5, family):
    '''
    Input: md5 hash
    Output: dictionary with the metadata key-value pairs
    
    '''
    metadata = {}
    metadata['md5'] = md5
    metadata['family'] = family
    
    params = {'apikey': 'f2f9bb5419691e82420cc5cba8ec39285eeec5a3a1bed5a459d79f66cc542b62',
              'resource': md5, 'allinfo': 1}
    headers = {
        "Accept-Encoding": "gzip, deflate",
        "User-Agent": "gzip,  My Python requests library example client or username"
    }
    response = requests.get('https://www.virustotal.com/vtapi/v2/file/report',
                            params=params, headers=headers)
    json_response = response.json()
    for key in json_response:
        print key
#     json.dumps(json_response)
    # if no metadata present, just skip the md5
    if 'additional_info' not in json_response:
        return {} 
    try:
        for f in json_response['additional_info']['exiftool']:
            metadata[f] = json_response['additional_info']['exiftool'][f]
    except KeyError:
        print("Invalid JSON Info")
        return {}
    print(metadata)
    return metadata

In [115]:
feature_list = set() # set of features (exhaustive)
md5_data = [] # list of md5 dictionaries for temporary storage


for column in df.columns:
    print(column)
    malware_family = column
    md5_list = list(df[column])
    
    print(md5_list)
    
    for md5 in md5_list:
        print(md5)
        md5_api_report = load_requests(md5, malware_family)
        
        if len(md5_api_report) == 0:
            continue
            
        for m in md5_api_report:
            if m not in feature_list:
                feature_list.add(m)
                
        md5_data.append(md5_api_report)

# Store the metadata of all md5 in a dataframe
df_metadata = pd.DataFrame(columns=list(feature_list))
for idx in range(len(md5_data)):
    md5_dict = md5_data[idx]
    for f in feature_list:
        if f in md5_dict:
            df_metadata.loc[idx, f] = md5_dict[f]
        else:
            df_metadata.loc[idx, f] = np.nan # NaN for missing values
        
# df_metadata.to_csv('../../data/lab2/md5_lab2_feat.csv')
df_metadata.to_csv('/Volumes/GoogleDrive/My Drive/Spring 19/CSE 494 Labs/Homework 1/hw1_md5_feat.csv')
df_metadata.head()
        


CTB-locker
['c4b29d3345ed0b7a0b284764f3446396', 'aadf8bb8770d563aa27eadc57ef238e9', 'db0a2be5b0eb4603fada6e6f79f3d267', '3be529d7c05913ef0571a6f4fbf7d946', '555eaa028a55704f3200614a01d0a464', 'b4a012e465f845bc7ac8b441bec2d2c3', 'f87208a911d9d1a3793914a649dac97e', '78b22232b87e0598e8f6f584f1883c87', '6d0509c64d650c95b0e132567c063eb2', '497b225179a648c3fcec8773351991b8', '1011a4fa57954818d4e378a2af9fc3d6', '77fac4194a04d2bbd9b4503044f4250c', 'fa51284e30ae0547fec613aecf61191a', '5bf3e5258ec9efe29f92acbac924c451', 'cf98e53ef516ef190ff0039073a048dd', 'b6cdfdef61c8acbcb72b43547eb7aab4', 'a4c64aefb73d3623521d91d5d63cdb45', '01a466176732f7050a2853334f2752b8', '44351f134a282f98359766ca2cd03e14', '428285ac08507dcaefe93d269247506d', '73c570a78ca50abea3a623c84fa69daa', '33e3869b8f83e4f6e7cf57e37e243bd7', '442d975dbb126cbc2e67e87e3fbdb3e7', '18770c7b60ea583d6a7f24f87ac1ba79', '1bf842a82acfa384cff8cafcc70847fe', '2d1626395dd249c3f0945597c3f6d82a', '3f40a4f7e25909f83fb3c9eea43fc5a6', 'afd71522bec434c

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'CTB-locker', u'TimeStamp': u'2015:02:15 09:00:31+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'165888', u'LinkerVersion': u'9.0', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'44032', u'SubsystemVersion': u'5.0', u'EntryPoint': u'0x1d7cb', u'OSVersion': u'5.0', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'0', 'md5': 'f87208a911d9d1a3793914a649dac97e'}
78b22232b87e0598e8f6f584f1883c87
vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tag

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'CTB-locker', u'TimeStamp': u'2016:03:09 14:03:47+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'16384', u'LinkerVersion': u'6.5', u'ImageFileCharacteristics': u'No relocs, Executable, No line numbers, No symbols, 32-bit', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'49152', u'SubsystemVersion': u'4.2048', u'EntryPoint': u'0x23b6', u'OSVersion': u'4.5', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'49152', 'md5': '5bf3e5258ec9efe29f92acbac924c451'}
cf98e53ef516ef190ff0039073a048dd
vhash
submission_names
scan_date
first_seen
to

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'CTB-locker', u'TimeStamp': u'2016:03:09 14:03:47+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'16384', u'LinkerVersion': u'6.5', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'49152', u'SubsystemVersion': u'4.2048', u'EntryPoint': u'0x23b6', u'OSVersion': u'4.0', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'49152', 'md5': '442d975dbb126cbc2e67e87e3fbdb3e7'}
18770c7b60ea583d6a7f24f87ac1ba79
vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scan

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'CTB-locker', u'TimeStamp': u'2016:01:20 17:42:46+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'28672', u'LinkerVersion': u'8.0', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'753664', u'SubsystemVersion': u'4.0', u'EntryPoint': u'0x7396', u'OSVersion': u'4.2', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'0', 'md5': '54fde8486cc14f738f71cc14d1e696ac'}
bcfd0793e4f8839cb1cfbc9baafdce6a
vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags

...

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'xtreme', u'TimeStamp': u'1992:06:19 23:22:17+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'51712', u'LinkerVersion': u'2.25', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'14336', u'SubsystemVersion': u'4.0', u'EntryPoint': u'0xd0f4', u'OSVersion': u'4.0', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'0', 'md5': '163abb0c7ca93edd01f5854465cb64c4'}
17f3da74634e562e7fe19c6c4e4e2bc3
vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
aut

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'xtreme', u'TimeStamp': u'1992:06:19 23:22:17+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'51712', u'LinkerVersion': u'2.25', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'19456', u'SubsystemVersion': u'4.0', u'EntryPoint': u'0xd0f4', u'OSVersion': u'4.0', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'0', 'md5': '26b2868c417367221699b392fff022dd'}
281bc2f3049df79163377480656fc30e
vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
aut

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'xtreme', u'TimeStamp': u'1992:06:19 23:22:17+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'51712', u'LinkerVersion': u'2.25', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'14336', u'SubsystemVersion': u'4.0', u'EntryPoint': u'0xd0f4', u'OSVersion': u'4.0', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'0', 'md5': '32e73a98543f0f9ca9b2126f323c7e58'}
332704bce32a7d10aadca990c6fd24f4
vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
aut

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': 'xtreme', u'TimeStamp': u'1992:06:19 23:22:17+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'51712', u'LinkerVersion': u'2.25', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'14336', u'SubsystemVersion': u'4.0', u'EntryPoint': u'0xd0f4', u'OSVersion': u'4.0', u'ImageVersion': u'0.0', u'UninitializedDataSize': u'0', 'md5': '3ed74518ed6a875f533db40057be26f0'}
3ed813685d098c623ac97a5a50265647
vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
aut

fileversion Comments Template          LanguageCode InternalName SharedDoc  \
0         NaN      NaN      NaN                   NaN          NaN       NaN   
1         NaN      NaN      NaN                 Greek       Puzzle       NaN   
2         NaN      NaN      NaN                   NaN          NaN       NaN   
3         NaN      NaN      NaN  Chinese (Simplified)      Elgamal       NaN   
4         NaN      NaN      NaN  Chinese (Simplified)      Elgamal       NaN   

  Company ZipBitFlag Originalfilename Legalcopyright  ...  \
0     NaN        NaN              NaN            NaN  ...   
1     NaN        NaN              NaN            NaN  ...   
2     NaN        NaN              NaN            NaN  ...   
3     NaN        NaN              NaN            NaN  ...   
4     NaN        NaN              NaN            NaN  ...   

  CharactersWithSpaces    FileDescription FileCreateDate originalfilename  \
0                  NaN                NaN            NaN              NaN   
1                  NaN             Puzzle            NaN              NaN   
2                  NaN                NaN            NaN              NaN   
3                  NaN  Elgamal Microsoft            NaN              NaN   
4                  NaN  Elgamal Microsoft            NaN              NaN   

  ZipCompressedSize                          MachineType FileSubtype  \
0               NaN  Intel 386 or later, and compatibles         NaN   
1               NaN  Intel 386 or later, and compatibles           0   
2               NaN  Intel 386 or later, and compatibles         NaN   
3               NaN  Intel 386 or later, and compatibles           0   
4               NaN  Intel 386 or later, and compatibles           0   

  EntryPoint          ObjectFileType Paragraphs  
0     0x1718                     NaN        NaN  
1     0x5996  Executable application        NaN  
2     0x203e                     NaN        NaN  
3     0x30af  Executable application        NaN  
4     0x30af  Executable application        NaN  

[5 rows x 89 columns]

In [129]:
load_requests('c4b29d3345ed0b7a0b284764f3446396','')

vhash
submission_names
scan_date
first_seen
total
additional_info
size
scan_id
times_submitted
harmless_votes
verbose_msg
sha256
type
scans
tags
authentihash
unique_sources
positives
ssdeep
md5
permalink
sha1
resource
response_code
community_reputation
malicious_votes
ITW_urls
last_seen
{u'MIMEType': u'application/octet-stream', u'Subsystem': u'Windows GUI', u'MachineType': u'Intel 386 or later, and compatibles', 'family': '', u'TimeStamp': u'2016:05:15 09:01:46+01:00', u'FileType': u'Win32 EXE', u'PEType': u'PE32', u'CodeSize': u'14536704', u'LinkerVersion': u'6.0', u'FileTypeExtension': u'exe', u'InitializedDataSize': u'40960', u'SubsystemVersion': u'4.0', u'EntryPoint': u'0x1718', u'OSVersion': u'144.0', u'ImageVersion': u'11.0', u'UninitializedDataSize': u'0', 'md5': 'c4b29d3345ed0b7a0b284764f3446396'}


{u'CodeSize': u'14536704',
 u'EntryPoint': u'0x1718',
 u'FileType': u'Win32 EXE',
 u'FileTypeExtension': u'exe',
 u'ImageVersion': u'11.0',
 u'InitializedDataSize': u'40960',
 u'LinkerVersion': u'6.0',
 u'MIMEType': u'application/octet-stream',
 u'MachineType': u'Intel 386 or later, and compatibles',
 u'OSVersion': u'144.0',
 u'PEType': u'PE32',
 u'Subsystem': u'Windows GUI',
 u'SubsystemVersion': u'4.0',
 u'TimeStamp': u'2016:05:15 09:01:46+01:00',
 u'UninitializedDataSize': u'0',
 'family': '',
 'md5': 'c4b29d3345ed0b7a0b284764f3446396'}

In [130]:
columns_to_select = []

for column in df_metadata.columns:
    num_missing = (df_metadata[column]).isnull().sum()
#     print(column, num_missing)

    if num_missing < 10:
        columns_to_select.append(column)

df_metadata = df_metadata[columns_to_select]
df_metadata.to_csv('/Volumes/GoogleDrive/My Drive/Spring 19/CSE 494 Labs/Homework 1/hw1_md5_feat_filtered.csv')

# manually curate features
columns_feat = ['md5', 'EntryPoint', 'family', 'LinkerVersion', 'CodeSize', 'InitializedDataSize', 'Subsystem']
df_metadata = df_metadata[columns_feat]
        
df_metadata.to_csv('/Volumes/GoogleDrive/My Drive/Spring 19/CSE 494 Labs/Homework 1/hw1_md5_feat_v1.csv')

    

In [131]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [132]:
# Impute missing values grouped by each family
from sklearn.preprocessing import Imputer

feat_data_types = ['EntryPoint', 'LinkerVersion', 'CodeSize', 'InitializedDataSize', 'Subsystem', 'family']


families = list(set(df_metadata['family']))

df_imputed = pd.DataFrame(columns=df_metadata.columns)
for f in families:
    df_family = df_metadata[df_metadata['family'] == f]
    
    df_family = DataFrameImputer().fit_transform(df_family)
    
    df_imputed = df_imputed.append(df_family)
    
df_imputed.to_csv('/Volumes/GoogleDrive/My Drive/Spring 19/CSE 494 Labs/Homework 1/hw1_md5_feat_imputed.csv')
# print df_imputed.head(100)

In [133]:
from collections import defaultdict

def get_ngrams_dict(text, n):
    ngrams = defaultdict(int)
    for i in range(len(text)):
        ngrams[text[i:i+n]] += 1
    
    return ngrams

In [134]:
# N grams for hexadecimal entry point

from collections import Counter
import operator 

data_hex = df_imputed['EntryPoint'].tolist()

ngrams_dict = defaultdict(int)
for i in range(len(data_hex)):
    data_hex[i] = (data_hex[i])[2:]
    print data_hex[i]
    ngrams_curr = get_ngrams_dict(data_hex[i], 2)
    print ngrams_curr
    input = [ngrams_dict, ngrams_curr]
    ngrams_dict = sum((Counter(ng) for ng in input),  Counter())
    print i
sorted_ngrams = sorted(ngrams_dict.items(), key=operator.itemgetter(1))[:10]
#sorted_ngrams is the only textual feature and as a result is the only feature that ngrams is needed for



# print sorted_ngrams
# Take the first 10  most frequent n grams and make them binary features

7e4a
defaultdict(<type 'int'>, {u'a': 1, u'e4': 1, u'7e': 1, u'4a': 1})
0
7e4a
defaultdict(<type 'int'>, {u'a': 1, u'e4': 1, u'7e': 1, u'4a': 1})
1
7e4a
defaultdict(<type 'int'>, {u'a': 1, u'e4': 1, u'7e': 1, u'4a': 1})
2
7e4a
defaultdict(<type 'int'>, {u'a': 1, u'e4': 1, u'7e': 1, u'4a': 1})
3
7e4a
defaultdict(<type 'int'>, {u'a': 1, u'e4': 1, u'7e': 1, u'4a': 1})
4
7e4a
defaultdict(<type 'int'>, {u'a': 1, u'e4': 1, u'7e': 1, u'4a': 1})
5
1000
defaultdict(<type 'int'>, {u'10': 1, u'00': 2, u'0': 1})
6
bdf7
defaultdict(<type 'int'>, {u'bd': 1, u'df': 1, u'7': 1, u'f7': 1})
7
bdf7
defaultdict(<type 'int'>, {u'bd': 1, u'df': 1, u'7': 1, u'f7': 1})
8
3212
defaultdict(<type 'int'>, {u'2': 1, u'32': 1, u'12': 1, u'21': 1})
9
12f3f
defaultdict(<type 'int'>, {u'2f': 1, u'12': 1, u'f3': 1, u'3f': 1, u'f': 1})
10
1000
defaultdict(<type 'int'>, {u'10': 1, u'00': 2, u'0': 1})
11
335a
defaultdict(<type 'int'>, {u'33': 1, u'a': 1, u'5a': 1, u'35': 1})
12
335a
defaultdict(<type 'int'>, {u'33': 1, u'

251
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
252
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
253
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
254
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
255
88f4
defaultdict(<type 'int'>, {u'f4': 1, u'88': 1, u'8f': 1, u'4': 1})
256
88f4
defaultdict(<type 'int'>, {u'f4': 1, u'88': 1, u'8f': 1, u'4': 1})
257
8abc
defaultdict(<type 'int'>, {u'c': 1, u'8a': 1, u'ab': 1, u'bc': 1})
258
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
259
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
260
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
261
8abc
defaultdict(<type 'int'>, {u'c': 1, u'8a': 1, u'ab': 1, u'bc': 1})
262
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
263
d0f4
defaultdict(<type 'int'>, {u'f4': 1, u'0f': 1, u'4': 1, u'd0': 1})
264
d0f4
def

In [135]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler 

In [136]:
# df_imputed.reindex
# df_imputed




In [137]:
#calculates Euclidean distance
def _distance(a,b):
    return np.sqrt(((a-b)**2).sum())

In [138]:
#first, find the sum of the distances from each sample to its respective centroid for each cluster
#then, return the minimum value
def _nearest(clusters,x):
    return np.argmin([_distance(x,c) for c in clusters])

In [139]:
#running Kmeans
def runKmeans(X, n_clust, init_method, max_iter_num):
    kmeans = KMeans(n_clusters = n_clust, init = init_method, verbose = 0, tol = 1e-3, max_iter = max_iter_num, n_init = 10)
    kmeans.fit(X)
    
    return kmeans

In [140]:
def findIndexOfNearestCluster(model, df, schools_dict, school_name):
    for idx, s in schools_dict.items():
        if s == school_name:
            feat_school = df.iloc[idx].values

    clus_cent = model.cluster_centers_

    print(_nearest(clus_cent, feat_school.T))

In [243]:
# columns_to_select = ['LinkerVersion', 'CodeSize', 'InitializedDataSize', 'Subsystem']
columns_to_select = ['Subsystem']


print df_imputed
for d in df_imputed['Subsystem'].values:
    if(d == 'Windows GUI'):
#         print 'true'
        df_imputed['Subsystem'] = 1
    else:
#         print 'false'
        df_imputed['Subsystem'] = 0

# print df_imputed


X = df_imputed[columns_to_select].values
print X

# model = runKmeans(X, 4, 'random', 300)
model = runKmeans(X, 6, 'random', 100)
# print("Silhouette scores evaluation of K means on unscaled data ")
# print(silhouette_score(X, model.labels_)1

# plt.figure(figsize=(10, 7))  
# plt.scatter(X[:,1], X[:,2], c=model.labels_, cmap='rainbow') # Inertia -  the sum of squared distances of samples to the nearest cluster centre
# plt.show()

                                  md5 EntryPoint  family LinkerVersion  \
148  297529814d8d292594a1981fad30daa6     0x7e4a   locky           9.0   
149  b97ed89e814ad91338a6bdd5f7853566     0x7e4a   locky           9.0   
150  40849d82a14058cbc91d0ecea473d1d8     0x7e4a   locky           9.0   
151  254c5c13dd02a9fccbe2a40ac0b04355     0x7e4a   locky           9.0   
152  1d0687fe7c7c5591f1049ecb84e8cbd9     0x7e4a   locky           9.0   
153  0aa56c23cff79948f977ebd1a470b4ad     0x7e4a   locky           9.0   
154  85af16270f649b670cd255e6054d5388     0x1000   locky           2.5   
155  901ef89350b60a992e1a6c67a61dcdab     0xbdf7   locky          10.0   
156  fe4985beae55b054259bf14d3a3e50a7     0xbdf7   locky          10.0   
157  3b522c3e3fc6cf29a2c8c65a80f14a08     0x3212   locky           6.0   
158  252957f37b8bd7a57473eab5f1a65d5c    0x12f3f   locky           9.0   
159  a8344ecd79b1c1371526ee06a0a0fcd6     0x1000   locky           2.5   
160  ea8b3acdfb0c30156977897f693f9db6 

In [241]:
for i in range(df_imputed.shape[0]):
    df_imputed['cluster'][i] = model.labels_[i]

# print model.inertia_
cluster_zero = 0;
cluster_one = 0;
cluster_two = 0;
cluster_three = 0;
cluster_four = 0;
cluster_five = 0;

for m in model.labels_:
    if m == 0:
        cluster_zero+=1
    elif m == 1:
        cluster_one+=1
    elif m == 2:
        cluster_two+=1
    elif m==3:
        cluster_three+=1
    elif m==4:
        cluster_four+=1
    elif m==5:
        cluster_five+=1

print df_imputed['cluster'].values

print cluster_zero
print cluster_one
print cluster_two
print cluster_three
print cluster_four
print cluster_five

/Users/anaygupta/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
293
0
0
0
0
0


In [244]:
select = ['family', 'cluster']
df_measure = df_imputed[select]

locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 0:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 0'
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 0
Locky: 
45
Dridex: 
49
Cybergate: 
50
CTB-locker: 
49
Teslacrypt: 
50
xtreme: 
50


In [245]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 1:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 1'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 1
Locky: 
0
Dridex: 
0
Cybergate: 
0
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [246]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 2:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 2'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 2
Locky: 
0
Dridex: 
0
Cybergate: 
0
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [247]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 3:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
           
print 'Cluster 3'
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 3
Locky: 
0
Dridex: 
0
Cybergate: 
0
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [248]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 4:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 4'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 4
Locky: 
0
Dridex: 
0
Cybergate: 
0
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [249]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 5:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 5'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 5
Locky: 
0
Dridex: 
0
Cybergate: 
0
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [250]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'locky':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'Locky: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three: '
print three
print 'four: ' 
print four
print 'five: '
print five

Locky: 
zero
45
one
0
two: 
0
three: 
0
four: 
0
five: 
0


In [251]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'dridex':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'dridex: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three: '
print three
print 'four: ' 
print four
print 'five: '
print five

dridex: 
zero
49
one
0
two: 
0
three: 
0
four: 
0
five: 
0


In [252]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'cybergate':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'cybergate: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

cybergate: 
zero
50
one
0
two: 
0
three 
0
four: 
0
five: 
0


In [253]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'CTB-locker':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'CTB-locker: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

CTB-locker: 
zero
49
one
0
two: 
0
three 
0
four: 
0
five: 
0


In [254]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'Teslacrypt':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'Teslacrypt: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

Teslacrypt: 
zero
50
one
0
two: 
0
three 
0
four: 
0
five: 
0


In [237]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'xtreme':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'xtreme: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

xtreme: 
zero
0
one
2
two: 
3
three 
44
four: 
1
five: 
0


In [ ]:
cluster_map = pd.DataFrame()

In [260]:
import scipy.cluster.hierarchy as shc


In [261]:
# plt.figure(figsize=(10, 7))  
# plt.title("School Dendograms")  
# dend = shc.dendrogram(shc.linkage(df.values[:100], method='single'))  

from sklearn.cluster import AgglomerativeClustering


columns_to_select = ['LinkerVersion', 'CodeSize', 'InitializedDataSize', 'Subsystem']


data = df_imputed[columns_to_select].values

cluster = AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='single')  
cluster_fit = cluster.fit(data)  

print(silhouette_score(data, cluster.labels_))
print cluster.labels_

0.8009476121092515
[1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 5 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 3 1 1
 0 1 0 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [263]:
for i in range(df_imputed.shape[0]):
    df_imputed['cluster'][i] = cluster.labels_[i]

# print model.inertia_
cluster_zero = 0;
cluster_one = 0;
cluster_two = 0;
cluster_three = 0;
cluster_four = 0;
cluster_five = 0;

for m in cluster.labels_:
    if m == 0:
        cluster_zero+=1
    elif m == 1:
        cluster_one+=1
    elif m == 2:
        cluster_two+=1
    elif m==3:
        cluster_three+=1
    elif m==4:
        cluster_four+=1
    elif m==5:
        cluster_five+=1

print df_imputed['cluster'].values

print cluster_zero
print cluster_one
print cluster_two
print cluster_three
print cluster_four
print cluster_five

/Users/anaygupta/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 3 1 1
 0 1 0 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 3 1 1 1 1 1 1 5 1 1 4 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
3
283
1
3
2
1


In [264]:
select = ['family', 'cluster']
df_measure = df_imputed[select]

locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 0:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 0'
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 0
Locky: 
3
Dridex: 
0
Cybergate: 
0
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [265]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 1:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 1'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 1
Locky: 
40
Dridex: 
48
Cybergate: 
47
CTB-locker: 
48
Teslacrypt: 
50
xtreme: 
50


In [266]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 2:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 2'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 2
Locky: 
0
Dridex: 
1
Cybergate: 
0
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [267]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 3:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
           
print 'Cluster 3'
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 3
Locky: 
2
Dridex: 
0
Cybergate: 
1
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [268]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 4:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 4'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 4
Locky: 
0
Dridex: 
0
Cybergate: 
1
CTB-locker: 
1
Teslacrypt: 
0
xtreme: 
0


In [269]:
locky = 0
dridex = 0
cybergate = 0
ctb = 0
teslacrypt = 0
xtreme = 0

for d in df_measure.values:
    if d[1] == 5:
        if d[0] == 'locky':
            locky+=1
            
            
        elif d[0] == 'dridex':
            dridex+=1
            
            
        elif d[0] == 'cybergate':
            cybergate+=1
            
            
        elif d[0] == 'CTB-locker':
            ctb+=1
            
            
        elif d[0] == 'Teslacrypt':
            teslacrypt+=1
            
            
        elif d[0] == 'xtreme':
            xtreme+=1
           
print 'Cluster 5'            
print 'Locky: '
print locky
print 'Dridex: '
print dridex
print 'Cybergate: '
print cybergate
print 'CTB-locker: '
print ctb
print 'Teslacrypt: ' 
print teslacrypt
print 'xtreme: '
print xtreme

Cluster 5
Locky: 
0
Dridex: 
0
Cybergate: 
1
CTB-locker: 
0
Teslacrypt: 
0
xtreme: 
0


In [270]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'locky':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'Locky: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three: '
print three
print 'four: ' 
print four
print 'five: '
print five

Locky: 
zero
3
one
40
two: 
0
three: 
2
four: 
0
five: 
0


In [271]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'dridex':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'dridex: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three: '
print three
print 'four: ' 
print four
print 'five: '
print five

dridex: 
zero
0
one
48
two: 
1
three: 
0
four: 
0
five: 
0


In [272]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'cybergate':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'cybergate: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

cybergate: 
zero
0
one
47
two: 
0
three 
1
four: 
1
five: 
1


In [273]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'CTB-locker':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'CTB-locker: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

CTB-locker: 
zero
0
one
48
two: 
0
three 
0
four: 
1
five: 
0


In [274]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'Teslacrypt':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'Teslacrypt: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

Teslacrypt: 
zero
0
one
50
two: 
0
three 
0
four: 
0
five: 
0


In [275]:
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0

for d in df_measure.values:
    if d[0] == 'xtreme':
        if d[1] == 0:
            zero+=1
            
            
        elif d[1] == 1:
            one+=1
            
            
        elif d[1] == 2:
            two+=1
            
            
        elif d[1] == 3:
            three+=1
            
            
        elif d[1] == 4:
            four+=1
            
            
        elif d[1] == 5:
            five+=1
           
print 'xtreme: '
print 'zero'
print zero
print 'one'
print one
print 'two: '
print two
print 'three '
print three
print 'four: ' 
print four
print 'five: '
print five

xtreme: 
zero
0
one
50
two: 
0
three 
0
four: 
0
five: 
0
